In [1]:
import matplotlib.pyplot as plt
import numpy as np

import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt

In [3]:
print(tf.__version__)

2.3.1


In [5]:
training_set = tf.keras.preprocessing.image_dataset_from_directory(
    'D:/Docs/PCB/Training_Dataset',
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    seed=123,
    image_size=(110, 42),
    shuffle=True,
    validation_split = 0.2,
    subset="training"
)

Found 12000 files belonging to 8 classes.
Using 9600 files for training.


In [7]:
class_names = training_set.class_names
print(class_names)

In [12]:
data_iterator = training_set.as_numpy_iterator()
batch = data_iterator.next()
batch[0].shape

In [15]:
batch[1]

array([4, 3, 0, 0, 0, 0, 3, 1, 1, 4, 1, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       3, 0, 1, 1, 3, 0, 0, 0, 4, 0])

### Plot some images

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 2, 1)
imgplot = plt.imshow(batch[1][1])
plt.colorbar(ticks=[0.1, 0.3, 0.5, 0.7], orientation='horizontal')
ax = fig.add_subplot(1, 2, 2)
imgplot = plt.imshow(batch[1][1])
imgplot.set_clim(0.0, 0.7)
ax.set_title('After')
plt.colorbar(ticks=[0.1, 0.3, 0.5, 0.7], orientation='horizontal')

In [27]:
training_set = training_set.map(lambda x,y: (x / 255, y))

In [29]:
data_iterator.next()

(array([[[[4.60000000e+01, 1.07619047e+01, 1.90476179e-01],
          [4.84285736e+01, 2.01428566e+01, 1.04285717e+01],
          [7.10476227e+01, 2.81904755e+01, 2.09523811e+01],
          ...,
          [1.37095062e+02, 3.58094559e+01, 1.13809204e+01],
          [4.34285889e+01, 2.11428528e+01, 9.57142639e+00],
          [3.70000000e+01, 2.11904755e+01, 4.61904907e+00]],
 
         [[4.59411240e+01, 1.09385281e+01, 2.18181804e-01],
          [4.92181854e+01, 2.06727276e+01, 1.08857145e+01],
          [7.27896118e+01, 2.91982670e+01, 2.18874454e+01],
          ...,
          [1.36495926e+02, 3.64085884e+01, 1.12562437e+01],
          [4.35013161e+01, 2.13610344e+01, 9.49869919e+00],
          [3.68095245e+01, 2.11766224e+01, 4.40086746e+00]],
 
         [[4.56320343e+01, 1.18658009e+01, 3.63636315e-01],
          [5.33636360e+01, 2.34545460e+01, 1.32857151e+01],
          [8.19350662e+01, 3.44891777e+01, 2.67965374e+01],
          ...,
          [1.33350479e+02, 3.95540352e+01, 1.0601